In [17]:
import mediapipe as mp
import cv2
import csv
import numpy as np

In [18]:
#mediapipe kütüphanesi içerisindeki çizim yapma olanağı fonksiyonu 
#ve holistic modeli(vücut duruşunu analiz edebilen model.)
mp_drawing = mp.solutions.drawing_utils
mp_holistic= mp.solutions.holistic

In [36]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        results = holistic.process(image)
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            point_11 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].y * image.shape[0]))
            point_13 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].y * image.shape[0]))
            point_15 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y * image.shape[0]))
            
            # Draw circles at the specified points
            cv2.circle(image, point_11, 5, (0, 255, 0), -1)
            cv2.circle(image, point_13, 5, (0, 255, 0), -1)
            cv2.circle(image, point_15, 5, (0, 255, 0), -1)
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [34]:
landmarks = ['sinif']
for val in range (1, 4):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [35]:
with open('Sağ Kol Dataları.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [41]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        results = holistic.process(image)
       
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            point_11 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].y * image.shape[0]))
            point_13 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].y * image.shape[0]))
            point_15 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y * image.shape[0]))
            
            # Draw circles at the specified points
            cv2.circle(image, point_11, 5, (0, 255, 0), -1)
            cv2.circle(image, point_13, 5, (0, 255, 0), -1)
            cv2.circle(image, point_15, 5, (0, 255, 0), -1)
       
        try:
            landmarks = results.pose_landmarks.landmark
            point_11 = np.array([landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].z])              

            point_13 = np.array([landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].x,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].y,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].z])

            point_15 = np.array([landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z])
            
            total = np.concatenate((point_11, point_13, point_15))
            RHand_row = list(total)
            
          #
            row = RHand_row
            #class_name bizim sınıflandırmamız için kullanılan değer
            #açık ve kapalı olmak üzere iki tür sınıflandırma yapacağız. 
            #ilk önce kapalı olarak 1100 değer alındı
            #1000 tanede açık olmak üzere veri alındı.
            class_name = "acik"
            row.insert(0, class_name)
            
            # elde edilen değerler oluşturulan pose.csv dosyasına kaydedildi.
            with open('Sağ Kol Dataları.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
                
        except Exception as errror:
            print(errror)
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()